In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from datetime import datetime
from IPython.display import display, Markdown
from pathvalidate import sanitize_filename

import utils

from prompts import Job_Post, Resume_Builder

In [32]:
## Inputs
my_files_dir = "my_applications"  # location for all job and resume files
job_file = "job.txt"  # filename with job post text. The entire job post can be pasted in this file, as is.
raw_resume_file = "resume_raw.yaml"  # filename for raw resume yaml. See example in repo for instructions.

# Model for matching resume to job post (except extraction chains)
# gpt-4 is not publicly available yet and can be 20-30 times costlier than the default model gpt-3.5-turbo.
# Simple extraction chains are hardcoded to use the cheaper gpt-3.5 model.
openai_model_name = "gpt-4"
openai_model_name = "gpt-3.5-turbo"

# temperature lower than 1 is preferred. temperature=0 returns deterministic output
temperature = 0.5

llm_kwargs = dict(
    model_name=openai_model_name,
    model_kwargs=dict(top_p=0.6, frequency_penalty=0.1),
)

In [33]:
# Step 1 - Read and parse job posting
job_post = Job_Post(
    utils.read_jobfile(os.path.join(my_files_dir, job_file)),
)
parsed_job = job_post.parse_job_post(verbose=False)

company_name = parsed_job["company"]
job_title = parsed_job["job_title"]
today_date = datetime.today().strftime("%Y%m%d")
job_filename = os.path.join(
    my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}")
)
print(f"#filename: {job_filename}.job\n")
utils.write_yaml(parsed_job, filename=f"{job_filename}.job")

#filename: my_applications\20230917__Artlume__Frontend Developer (ReactJS & VueJS).job



In [34]:
# Step 2 - read raw resume and create Resume builder object
my_resume = Resume_Builder(
    resume=utils.read_yaml(filename=os.path.join(my_files_dir, raw_resume_file)),
    parsed_job=parsed_job,
    llm_kwargs=llm_kwargs,
)
projects = my_resume.projects_raw
experiences = my_resume.experiences_raw
skills = my_resume.skills_raw

In [35]:
def update_resume_data(edits):
    # Review the generated output in previous cell.
    # If any updates are needed, copy the cell output below between the triple quotes
    # Set value to """" """" if no edits are needed
    edits = edits.strip()
    updated = []
    if edits:
        new_edit = utils.read_yaml(edits)
        if "experiences" in new_edit:
            updated.append('experiences')
            my_resume.experiences = new_edit["experiences"]
        if "projects" in new_edit:
            updated.append('projects')
            my_resume.projects = new_edit["projects"]
        if "skills" in new_edit:
            updated.append('projects')
            my_resume.skills = new_edit["skills"]
        if "summary" in new_edit:
            updated.append('summary')
            my_resume.summary = new_edit["summary"]

    print(f"Successfully: {', '.join(updated)} updated successfully")

In [37]:
experiences = my_resume.rewrite_unedited_experiences(verbose=False)
my_resume.experiences = experiences

experiences_yaml = utils.dict_to_yaml_string(dict(experiences=experiences))
update_resume_data(experiences_yaml)

SHANGHAI FRIENDESS ELECTRONICS TECHNOLOGY. Start: 
 ['Cloudnest. Contributed to the development of a web-based nesting software for plate laser cutting systems, where I implemented key features such as Report Generation, Label Printing, and Rule Identification. I designed and built the Report generation feature, enabling users to generate over 500 customized PDF and Excel reports using various editing tools. This resulted in a 30% increase in user productivity. I also developed the Label Printing feature, which streamlined production by allowing manufacturers to quickly print labels for each part of the nesting result.', 'MetalVerse. Played a critical role in the development of an automatic 3D model and CAD drawing generation system used for kitchen production. I developed key features such as Cabinet Generation, Board Hole Digging, and House Type Generation using Babylon.js and my knowledge of graphics and JavaScript to ensure the system was user-friendly and efficient.', 'WebCAD. Des

In [38]:
experiences

[{'company': 'SHANGHAI FRIENDESS ELECTRONICS TECHNOLOGY.',
  'startdate': 2021.07,
  'enddate': 2022.08,
  'titles': [{'name': 'Web Front-end Development Engineer', 'depart': 'Software R&D Department', 'startdate': 2021.07, 'enddate': 2022.08}],
  'highlights': ['Highlight 1: Developed a web-based nesting software for plate laser cutting systems at Cloudnest, enabling users to generate over 500 customized PDF and Excel reports using various editing tools. Increased user productivity by 30% and streamlined production by implementing features like Report Generation and Label Printing.',
   'Highlight 2: Played a critical role in MetalVerse, where I developed an automatic 3D model and CAD drawing generation system for kitchen production. Implemented features like Cabinet Generation and Board Hole Digging using Babylon.js and improved user satisfaction scores by 15%.',
   "Highlight 3: Led the development of a Material Calculation System, reducing production time by 40% through the use of 

In [39]:
# Step 4 - Rephrase projects
projects = my_resume.rewrite_projects_desc(verbose=False)
my_resume.projects = projects

projects_yaml = utils.dict_to_yaml_string(dict(projects=projects))
update_resume_data(projects_yaml)

Portfolio Start: 
 None
Portfolio End 0.000000: 
 ['Developed a personal portfolio using native Web front-end technology and Python back-end, showcasing expertise in both front-end and back-end development.']
Sonder Notebook Start: 
 None
Sonder Notebook End 0.000000: 
 ['An online note-taking application developed based on Docsify and Markdown, using Github as warehouse storage and Github Action as an automated deployment tool.']
Virtual Library Homepage Start: 
 None
Virtual Library Homepage End 0.000000: 
 ['Developed a ReactJS library management system for the web-side of a Unity project, demonstrating familiarity with various web development technologies.']
Successfully: projects updated successfully


In [41]:
# Step 5 - Extract skills
# This will match the required skills from the job post with your resume sections
# Outputs a combined list of skills extracted from the job post and included in the raw resume
skills = my_resume.extract_matched_skills(verbose=False)
my_resume.skills = skills
# skills_yaml = utils.dict_to_yaml_string(dict(skills=skills))
# update_resume_data(skills_yaml)

skills

[{'category': 'Technical',
  'skills': ['React.js',
   'Vue.js',
   'JavaScript',
   'CSS',
   'HTML5',
   'Git',
   'React',
   'Vue',
   'Npm',
   'Webpack',
   'Node.js',
   'Python',
   'Flask',
   'Devops with Github workflow',
   'C++',
   'openGL',
   'Blender',
   'Unity',
   'Unreal Engine 5',
   'Photoshop',
   'Premiere',
   'Adobe XD',
   'Figma']},
 {'category': 'Non-technical',
  'skills': ['Problem-solving',
   'Attention to detail',
   'Collaboration',
   'Teamwork',
   'Communication',
   'Memory and Recall',
   'Autonomy',
   'Written Communication',
   'Verbal Communication',
   'Time Management',
   'Diversity and Inclusion']}]

In [44]:
# Step 6 - Create a resume summary

summary = my_resume.write_summary(verbose=True)
my_resume.summary = summary
# summary_yaml = utils.dict_to_yaml_string(dict(summary=summary))
# update_resume_data(summary_yaml)



> Entering new  chain...
Prompt after formatting:
System: You are an expert technical writer. Your goal is to strictly follow all the provided <Steps> and meet all the given <Criteria>.
Human: <Job Posting>
Artlume
We3World Ltd is looking for an experienced Frontend Developer specializing in ReactJS and VueJS to join our UK-based team and help us shape the future of digital experiences.
Human: <Resume>
Education:
- 

Experience:
- 

- 1 years experience in:
  - Highlight 1: Developed a web-based nesting software for plate laser cutting systems at Cloudnest, enabling users to generate over 500 customized PDF and Excel reports using various editing tools. Increased user productivity by 30% and streamlined production by implementing features like Report Generation and Label Printing.
  - Highlight 2: Played a critical role in MetalVerse, where I developed an automatic 3D model and CAD drawing generation system for kitchen production. Implemented features like Cabinet Generation and Boar

In [45]:
summary

'Professional Summary:\nWith 1 year of experience specializing in ReactJS and VueJS, I am an accomplished Frontend Developer known for my strong work ethic and attention to detail. At Cloudnest, I developed a web-based nesting software that increased user productivity by 30% and streamlined production through features like Report Generation and Label Printing. As a part of your team at Artlume, I aim to shape the future of digital experiences by leveraging my expertise in ReactJS, VueJS, and my ability to collaborate effectively with backend developers, designers, and product managers.'

In [46]:
# Step 7 - Generate final resume yaml for review
my_resume.summary = summary
my_resume.experiences = experiences
my_resume.projects = projects
my_resume.skills = skills

today_date = datetime.today().strftime("%Y%m%d")
resume_filename = os.path.join(
    my_files_dir, sanitize_filename(f"{today_date}__{company_name}__{job_title}")
)
resume_final = my_resume.finalize()
print(f"#filename: {resume_filename}.yaml\n")
utils.write_yaml(resume_final, filename=f"{resume_filename}.yaml")

#filename: my_applications\20230917__Artlume__Frontend Developer (ReactJS & VueJS).yaml



In [47]:
# Step 8 - Identify resume improvements
# A previously generated resume can also be used here by manually providing resume_filename. Requires the associated parsed job file.
final_resume = Resume_Builder(
    resume=utils.read_yaml(filename=f"{resume_filename}.yaml"),
    parsed_job=utils.read_yaml(filename=f"{resume_filename}.job"),
    is_final=True,
    llm_kwargs=llm_kwargs,
)
improvements = final_resume.suggest_improvements(verbose=True)
improvements_yaml = utils.dict_to_yaml_string(dict(improvements=improvements))



> Entering new  chain...
Prompt after formatting:
System: You are an expert critic. Your goal is to strictly follow all the provided <Steps> and meet all the given <Criteria>.
Human: <Job Posting>
The ideal candidate is able to perform the following duties:
- Design, develop, and implement user-friendly web applications using ReactJS, VueJS, and other modern web technologies.
- Collaborate with backend developers, designers, and product managers to create seamless and visually appealing user interfaces.
- Translate wireframes and designs into high-quality code, ensuring optimal performance across all platforms and devices.
- Write clean, maintainable, and reusable code following best practices.
- Optimize applications for maximum speed and scalability.
- Conduct thorough testing to identify and fix bugs and improve usability.
- Provide technical support and guidance to other team members, fostering a culture of knowledge sharing and collaboration.
- Stay up to date with the latest in


> Finished chain.


In [49]:
summary

'Professional Summary:\nWith 1 year of experience specializing in ReactJS and VueJS, I am an accomplished Frontend Developer known for my strong work ethic and attention to detail. At Cloudnest, I developed a web-based nesting software that increased user productivity by 30% and streamlined production through features like Report Generation and Label Printing. As a part of your team at Artlume, I aim to shape the future of digital experiences by leveraging my expertise in ReactJS, VueJS, and my ability to collaborate effectively with backend developers, designers, and product managers.'

In [50]:
# Step 9 - Generate pdf from yaml
# Most common errors during pdf generation occur due to special characters. Escape them with backslashes in the yaml, e.g. $ -> \$
# pdf_file = utils.generate_pdf(yaml_file=f"{resume_filename}.yaml")
# display(Markdown((f"[{pdf_file}](<{pdf_file}>)")))

utils.generate_new_tex(yaml_file=f"{resume_filename}.yaml")

Done


In [55]:
print(f"{resume_filename}.yaml")
yaml_data = utils.read_yaml(filename=f"{resume_filename}.yaml")

my_applications\20230917__Artlume__Frontend Developer (ReactJS & VueJS).yaml


In [56]:
yaml_data

{'basic': {'name': 'Long Pan', 'address': 'Dublin, Ireland', 'label': 'Frontend Developer (ReactJS & VueJS)', 'contact': {'email': 'panlo@tcd.ie', 'phone': '(+353) 087-4615379'}, 'websites': [{'text': 'chenmo1212.cn', 'url': 'https://www.chenmo1212.cn?f=cv', 'icon': 'globe'}, {'text': 'Long Pan', 'url': 'https://www.linkedin.com/in/long-pan-chandler/', 'icon': 'linkedin'}, {'text': 'Chenmo1212', 'url': 'https://github.com/Chenmo1212?f=cv', 'icon': 'github'}]}, 'summary': 'Professional Summary:\nWith 1 year of experience specializing in ReactJS and VueJS, I am an accomplished Frontend Developer known for my strong work ethic and attention to detail. At Cloudnest, I developed a web-based nesting software that increased user productivity by 30% and streamlined production through features like Report Generation and Label Printing. As a part of your team at Artlume, I aim to shape the future of digital experiences by leveraging my expertise in ReactJS, VueJS, and my ability to collaborate e

In [62]:
from yaml_to_json import yaml_to_json
import json
import re

def generate_json():
    yaml_data = utils.read_yaml(filename=f"{resume_filename}.yaml")
    json_data = yaml_to_json(yaml_data)

    # Write the data to the JSON file
    with open(f"{resume_filename}.json", "w", encoding="utf-8") as json_file:
        json.dump(json_data, json_file)

    print("Successfully generate json file.")

generate_json()

Successfully generate json file.
